# HW2 - DENSER-Like Algorithms

## Import Libraries

In [1]:
from IPython.display import display, HTML
import cv2
import os
import random
import warnings 
import numpy as np
import pandas as pd
import seaborn
import torch.nn as nn
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from keras.layers import BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import datasets, Sequential, layers, models, initializers, regularizers, optimizers, metrics, utils
from tensorflow.keras import models
from keras_tuner import RandomSearch
from tensorflow.keras.utils import load_img, img_to_array
from PIL import ImageFile
import tensorflow.keras.utils  as utils
from IPython.display import display
import pydot
from sklearn.model_selection import train_test_split

## Definition of the string-based genome

In [2]:
#Defining a string-based representation for the networks (the genotype)
layers_and_activations = [
    ('Linear', [('number_of_features', 1, 512), ('bias', 0.0, 1.0)]),
    ('BatchNorm1d', [('num_features', 1, 512),('eps', 1e-5, 1e-2), ('momentum', 0.1, 0.99)]),
    ('LayerNorm', [('normalized_shape', 1, 512), ('eps', 1e-5, 1e-2)]),
    ('Dropout', [('p', 0.0, 1.0)]),
    ('AlphaDropout', [('p', 0.0, 1.0)]),
    ('Activation', ['Sigmoid', 'ReLU', 'PReLU', 'ELU', 'SELU', 'GELU', 'CELU', 'SiLU']),
]


def select_random_params(layer_tuple):
    """
    Selects random parameters within the provided range for a given layer.
    """
    layer_name, param_list = layer_tuple
    random_params = []

    # For 'Activation', we just want to randomly select one from the list
    if layer_name == 'Activation':
        random_params.append(('function', random.choice(param_list)))
    else:
        for param_tuple in param_list:
            if isinstance(param_tuple[1], int):  # integer parameter
                random_value = random.randint(param_tuple[1], param_tuple[2])
            else:  # float parameter
                random_value = random.uniform(param_tuple[1], param_tuple[2])
                random_value = round(random_value, 6)  # rounding to keep the number manageable

            random_params.append((param_tuple[0], random_value))

    return layer_name, random_params

# Test the function with each layer
for layer in layers_and_activations:
    print(f"{select_random_params(layer)}")


def generate_random_model_spec(input_features, output_features):
    model_spec = []

    # Input layer
    layer_name, layer_params = select_random_params(layers_and_activations[0])
    layer_params_dict = dict(layer_params)  # Convert the list of tuples to a dictionary
    last_out_features = layer_params_dict['number_of_features']
    model_spec.append(('Linear', {'in_features': input_features, 'out_features': last_out_features}))

    # Hidden layers
    for i in range(random.randint(1, 5)):  # Choose a random number of layers to add
        layer_choice = random.choice(layers_and_activations)
        layer_name, layer_params = select_random_params(layer_choice)
        layer_params_dict = dict(layer_params)  # Convert the list of tuples to a dictionary
        if layer_name == 'Linear':
            last_out_features = layer_params_dict['number_of_features']
            layer_params_dict = {'in_features': last_out_features, 'out_features': layer_params_dict['number_of_features']}
        elif layer_name in ['BatchNorm1d']:
            layer_params_dict = {'num_features': last_out_features}
        elif layer_name in ['LayerNorm']:
            layer_params_dict = {'normalized_shape': last_out_features}
        model_spec.append((layer_name, layer_params_dict))

    # Output layer
    model_spec.append(('Linear', {'in_features': last_out_features, 'out_features': output_features}))

    return model_spec

('Linear', [('number_of_features', 268), ('bias', 0.737495)])
('BatchNorm1d', [('num_features', 227), ('eps', 0.000529), ('momentum', 0.495779)])
('LayerNorm', [('normalized_shape', 100), ('eps', 0.006907)])
('Dropout', [('p', 0.115773)])
('AlphaDropout', [('p', 0.704755)])
('Activation', [('function', 'SiLU')])


## Defining a network class that is able to parse those instructions and build a functional pytorch network structure (the phenotype).

In [3]:
#Each network needs to have at least 1 layer and a maximum 50 layers. This choice
#needs to be made at random when generating the genotype.


class DeepNetwork:
    def __init__(self, genotype, phenotype):
        self.genotype
        self.phenotype
    
    def set_genotype(self, genotype_value):
        self.genotype = genotype_value

    def set_phenotype(self, phenotype_value):
        self.phenotype = phenotype_value

    def genotype_to_phenotype(genotype):
        #Tenho de construir aqui o transformador de g para f        
        self.genotype = genotype_value
        return genotype
    
    
import random


def create_model_from_spec(model_spec):
    model = nn.Sequential()

    for i, (layer_name, layer_params) in enumerate(model_spec):
        if layer_name == 'Activation':
            activation_function = getattr(nn, layer_params['function'])()
            model.add_module(f'Activation{i}', activation_function)
        else:
            layer = getattr(nn, layer_name)(**layer_params)
            model.add_module(f'{layer_name}{i}', layer)

    return model


# Test the functions
random_model_spec = generate_random_model_spec(input_size, 10)
print(random_model_spec)

random_model = create_model_from_spec(random_model_spec)
print(random_model)

NameError: name 'input_size' is not defined

## Defining a string-based representation for the optimizer.

In [ ]:
#Defining a string-based representation for the networks (the genotype)
optimizers = [
    ('Adam', [('lr', -3, 0), ('beta1', 0.8, 0.9), ('beta2', 0.99, 0.999)]),
    ('AdamW', [('lr', -4, -1), ('beta1', 0.8, 0.9), ('beta2', 0.99, 0.999), ('weight_decay', -4, -1)]),
    ('Adadelta', [('lr', -3, 0), ('rho', 0.9, 0.99)]),
    ('NAdam', [('lr', -3, 0), ('beta1', 0.8, 0.9), ('beta2', 0.99, 0.999), ('momentum_decay', 0.003, 0.005)]),
    ('SGD', [('lr', -3, 0), ('momentum', 0.5, 0.99), ('nesterov', False, True)]),
]

def select_random_params_op(optimizer_tuple):
    """
    Selects random parameters within the provided range for a given optimizer.
    """
    optimizer_name, param_list = optimizer_tuple
    random_params = []

    for param_tuple in param_list:
        if param_tuple[0] == 'nesterov':  # boolean parameter
            random_value = random.choice([param_tuple[1], param_tuple[2]])
        else:  # float parameter
            random_value = 10 ** random.uniform(param_tuple[1], param_tuple[2])
            random_value = round(random_value, 6)  # rounding to keep the number manageable

        random_params.append((param_tuple[0], random_value))

    return optimizer_name, random_params

# Test the function with each optimizer
for optimizer in optimizers:
    print(f"{select_random_params(optimizer)}")
    

## Defining a way to parse those instructions and build and functional pytorch optimizer.

In [49]:
class DeepOptimizers:
    def __init__(self, genotype=None, phenotype=None):
        if genotype is None:
            self.genotype = genotype
            self.phenotype = phenotype
        else:
            self.genotype = genotype
            self.phenotype = phenotype
    
    def set_genotype(self, genotype_value):
        self.genotype = genotype_value

    def set_phenotype(self, phenotype_value):
        self.phenotype = phenotype_value

    def genotype_to_phenotype(genotype):
        #Tenho de construir aqui o transformador de g para f        
        self.genotype = genotype_value
        return genotype

## Sample all parameter values from a grammar (this allows for a restricted search space and removes the need to deal with invalid combinations).

### Rules
1. BatchNorm vs LayerNorm
2. Dropout vs AlphaDropout
3. Dropout or Norm after Linear

## Define 4 simple genetic operators:
1. Network crossover
2. Add layer mutation
3. Remove layer mutation
4. Change optimizer mutation

## Experimental setup

In [50]:
def initialize_population(genotype=None):
    return ([DeepNetwork(genotype=genotype) for _ in range(5)])

def run(self, selection=0, crossover=0, mutation=0, metric="manhattan"):
    self.initialize_population(policy_matrix=None)

    for gen in range(self.generations):
        for agent in self.population:
            self.evaluate_fitness(agent, metric)



In [51]:
def create_random_model(input_features, output_features):
    model = nn.Sequential()

    # Input layer
    layer_name, layer_params = select_random_params(('Linear', [('number_of_features', 1, 512), ('bias', 0.0, 1.0)]))
    layer_params_dict = dict(layer_params)  # Convert the list of tuples to a dictionary
    last_out_features = layer_params_dict['number_of_features']
    model.add_module('InputLayer', nn.Linear(input_features, last_out_features))
    model.add_module('InputActivation', nn.ReLU())  # Adding an activation function

    # Hidden layers
    for i in range(random.randint(1, 50)):  # Choose a random number of layers to add
        layer_choice = random.choice(layers_and_activations)
        if layer_choice[0] != 'Linear':  # We don't want to start a hidden layer with anything other than a Linear layer
            continue
        layer_name, layer_params = select_random_params(layer_choice)
        layer_params_dict = dict(layer_params)  # Convert the list of tuples to a dictionary
        model.add_module('HiddenLinear' + str(i), nn.Linear(last_out_features, layer_params_dict['number_of_features']))
        last_out_features = layer_params_dict['number_of_features']  # Update the last output features count
        model.add_module('HiddenActivation' + str(i), nn.ReLU())  # Adding an activation function
        
        # Optionally add a dropout or normalization layer
        if random.choice([True, False]):
            layer_choice = random.choice(layers_and_activations)
            if layer_choice[0] in ['Dropout', 'AlphaDropout']:
                layer_name, layer_params = select_random_params(layer_choice)
                layer_params_dict = dict(layer_params)
                model.add_module('Hidden' + layer_name + str(i), getattr(nn, layer_name)(**layer_params_dict))
            elif layer_choice[0] in ['BatchNorm1d','LayerNorm']:
                layer_name, layer_params = select_random_params(layer_choice)
                layer_params_dict = dict(layer_params)
                model.add_module('Hidden' + layer_name + str(i), getattr(nn, layer_name)(last_out_features, **layer_params_dict))

    # Output layer
    model.add_module('OutputLayer', nn.Linear(last_out_features, output_features))

    return model



# Test the function
random_model = create_random_model(input_size, 10)
print(random_model)

Sequential(
  (InputLayer): Linear(in_features=784, out_features=248, bias=True)
  (InputActivation): ReLU()
  (HiddenLinear5): Linear(in_features=248, out_features=63, bias=True)
  (HiddenActivation5): ReLU()
  (OutputLayer): Linear(in_features=63, out_features=10, bias=True)
)


In [52]:
def create_random_optimizer(model_parameters):
    # Choose a random optimizer
    optimizer_choice = random.choice(optimizers)

    # Get random parameters for the chosen optimizer
    optimizer_name, optimizer_params = select_random_params_op(optimizer_choice)
    optimizer_params_dict = dict(optimizer_params)  # Convert the list of tuples to a dictionary

    # Create the optimizer
    optimizer = getattr(torch.optim, optimizer_name)(model_parameters, **optimizer_params_dict)

    return optimizer


In [65]:
random_optimizer = create_random_optimizer(random_model.parameters())
print(random_optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.024454
    maximize: False
    momentum: 4.342836
    nesterov: False
    weight_decay: 0
)


In [93]:
loss_fn = nn.CrossEntropyLoss()

num_epochs = 5
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.view(x_batch.size(0), -1)  # flatten the input
        pred = random_model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        random_optimizer.step()
        random_optimizer.zero_grad()
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_loader.dataset)
    print(f'Epoch {epoch}  Accuracy {accuracy_hist_train:.4f}')


Epoch 0  Accuracy 0.1063
Epoch 1  Accuracy 0.1057
Epoch 2  Accuracy 0.1046
Epoch 3  Accuracy 0.1075
Epoch 4  Accuracy 0.1041
